# COCO data preprocessing

This code will download the caption anotations for coco and preprocess them into an hdf5 file and a json file. 

These will then be read by the COCO data loader in Lua and trained on.

In [1]:
# lets download the annotations from http://mscoco.org/dataset/#download
import os
os.system('wget http://msvocds.blob.core.windows.net/annotations-1-0-3/captions_train-val2014.zip') # ~19MB

0

In [2]:
os.system('unzip captions_train-val2014.zip')

0

In [9]:
import json
val = json.load(open('annotations/val_hindi.json', 'r'))
infoOfVal = json.load(open('annotations/captions_val2014.json', 'r'))
infoOfVal = infoOfVal['images']
val['images'] = infoOfVal
#train = json.load(open('annotations/captions_train2014.json', 'r'))

In [10]:
print( val.keys() )
#print( val['info'] )
print( len(val['images']) )
print( len(val['annotations']) )
#print( val['images'][0] )
print( val['annotations'][0] )

dict_keys(['annotations', 'images'])
40504
39792
{'caption': ['स्केटबोर्डर पिकनिक तालिका का उपयोग करके अपने मंच के रूप में शो पर डाल रहा है।', 'एक स्केटबोर्डर एक पिकनिक तालिका के शीर्ष पर चालें खींच रहा है', 'एक मेज पर एक स्केटबोर्ड पर सवार एक आदमी', 'एक स्केटबोर्डर एक पिकनिक तालिका पर एक चाल कर रहा है', 'एक व्यक्ति पिकनिक की मेज पर एक स्केटबोर्ड की सवारी कर रहा है जिसमें भीड़ को देख कर देखा जा सकता है।'], 'image_id': 262148}


In [7]:
import json
import os

# combine all images and annotations together
imgs = val['images']#+ train['images']
annots = val['annotations']# + train['annotations']

# for efficiency lets group annotations by image
itoa = {}
for a in annots:
    imgid = a['image_id']
    if not imgid in itoa: itoa[imgid] = []
    itoa[imgid].append(a)

# create the json blob
out = []
for i,img in enumerate(imgs):
    imgid = img['id']
    
    # coco specific here, they store train/val images separately
    loc = 'train2014' if 'train' in img['file_name'] else 'val2014'
    
    jimg = {}
    jimg['file_path'] = os.path.join(loc, img['file_name'])
    jimg['id'] = imgid
    
    sents = []
    annotsi = itoa[imgid]
    for a in annotsi:
        sents.append(a['caption'])
    jimg['captions'] = sents
    out.append(jimg)
    
json.dump(out, open('coco_raw.json', 'w'))

KeyError: 'images'

In [7]:
# lets see what they look like
print(out[0])

{'captions': [u'A man with a red helmet on a small moped on a dirt road. ', u'Man riding a motor bike on a dirt road on the countryside.', u'A man riding on the back of a motorcycle.', u'A dirt path with a young person on a motor bike rests to the foreground of a verdant area with a bridge and a background of cloud-wreathed mountains. ', u'A man in a red shirt and a red hat is on a motorcycle on a hill side.'], 'file_path': u'val2014/COCO_val2014_000000391895.jpg', 'id': 391895}
